In [3]:
import ast
input_file1 = "QM9_129440_MLtraining"
input_file2 = "QM9_49762_MLtraining"
targets1 = ['HOMO_eV', 'LUMO_eV', 'c_v']
targets2 = ['bps_pred', 'mps_pred', 'fps_pred']
# ========= Cấu hình trọng số =========
WEIGHTS = {
    'RF_score_rank': 0.2,
    'XGB_score_rank': 0.2,
    'SKB_score_rank': 0.3,
    'PCA_score_rank': 0.3,
    'FS_BIC_score_rank': 4
}

In [4]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression  # Thêm import
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import pickle
import os
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from sklearn.metrics import make_scorer

df2 = pd.read_csv(f'{input_file2}.csv').replace([np.inf, -np.inf], np.nan).dropna()
df1 = pd.read_csv(f'{input_file1}.csv').replace([np.inf, -np.inf], np.nan).dropna()
epsilon = 1e-6
for col in targets1:
    if col in df1.columns:
        df1.loc[df1[col] == 0, col] = epsilon
    if col in df2.columns:
        df2.loc[df2[col] == 0, col] = epsilon
final_features = {}
def calculate_weighted_rank(row):
    return sum(row[method] * WEIGHTS[method] for method in WEIGHTS.keys())

def calculate_bic_score(model, X, y):
    n = len(y)
    k = X.shape[1] + 1  # +1 for intercept
    y_pred = model.predict(X)
    mse = ((y - y_pred) ** 2).mean()
    bic = n * np.log(mse) + k * np.log(n)
    return bic

def forward_selection_with_bic(X, y):
    remaining_features = list(X.columns)
    selected_features = []
    current_bic = None
    
    while remaining_features:
        bic_candidates = []
        for feature in remaining_features:
            candidate_features = selected_features + [feature]
            X_candidate = X[candidate_features]
            model = LinearRegression()
            model.fit(X_candidate, y)
            bic = calculate_bic_score(model, X_candidate, y)
            bic_candidates.append((bic, feature))
        
        bic_candidates.sort()
        best_bic, best_feature = bic_candidates[0]
        
        if current_bic is None or best_bic < current_bic:
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)
            current_bic = best_bic
        else:
            break
            
    return selected_features

def calculate_weighted_rank(row):
    return sum(row[method] * WEIGHTS[method] for method in WEIGHTS.keys())
open("selected_feature.txt", "w", encoding="utf-8").close()
open("combined.txt", "w", encoding="utf-8").close()

for target in targets2:
    X = df2.drop(columns=['bps_pred', 'mps_pred', 'fps_pred', 'canonical_smiles'])
    y = df2[target]
    rf = RandomForestRegressor(n_estimators=200, random_state=7)
    rf.fit(X, y)
    rf_imp = pd.DataFrame({'feature': X.columns, 'RF_score': rf.feature_importances_})
    xgb = XGBRegressor(n_estimators = 1000, random_state=7)
    xgb.fit(X, y)
    xgb_imp = pd.DataFrame({'feature': X.columns, 'XGB_score': xgb.feature_importances_})
    selector = SelectKBest(mutual_info_regression, k='all')
    selector.fit(X, y)
    skb_imp = pd.DataFrame({'feature': X.columns, 'SKB_score': selector.scores_})
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    pca = PCA(n_components=0.9999)
    pca.fit(X_scaled)
    pca_loadings = pd.DataFrame(pca.components_.T, index=X.columns).abs().sum(axis=1)
    pca_imp = pd.DataFrame({'feature': pca_loadings.index, 'PCA_score': pca_loadings.values})
    selected_fs_bic = forward_selection_with_bic(X, y)
    fs_bic_scores = {feature: 1.0 for feature in selected_fs_bic}
    fs_bic_imp = pd.DataFrame({
        'feature': X.columns,
        'FS_BIC_score': [fs_bic_scores.get(f, 0.0) for f in X.columns]  # 1.0 nếu được chọn, 0.0 nếu không
    })

    combined = rf_imp.merge(skb_imp, on='feature')\
                    .merge(pca_imp, on='feature')\
                    .merge(xgb_imp, on='feature')\
                    .merge(fs_bic_imp, on='feature')


    rank_columns = []
    for method in ['RF_score', 'SKB_score', 'PCA_score', 'XGB_score', 'FS_BIC_score']:
        ascending = False 
        combined[f'{method}_rank'] = combined[method].rank(ascending=ascending, method='dense')
        rank_columns.append(f'{method}_rank')
    
    combined['Weighted_Avg_Rank'] = combined[rank_columns].apply(
        lambda row: sum(row[col] * WEIGHTS.get(col, 0) for col in WEIGHTS.keys()), axis=1
    )
    with open("combined.txt", "a", encoding="utf-8") as file_out:
        file_out.write(f"\n=== combined dataframe of {target}===\n")
        file_out.write(combined.to_string())
        file_out.write(f"\n=== end of combined dataframe ===\n")
    
    top_features = combined.sort_values('Weighted_Avg_Rank').head(20)['feature'].tolist()
    final_features[target] = top_features
    with open("selected_feature.txt", "a", encoding="utf-8") as f:
        f.write(f"\n{target} : {top_features}\n")

for target in targets1:
    X = df1.drop(columns=['HOMO_eV', 'LUMO_eV', 'c_v', 'canonical_smiles'])
    y = df1[target]
    rf = RandomForestRegressor(n_estimators=200, random_state=7)
    rf.fit(X, y)
    rf_imp = pd.DataFrame({'feature': X.columns, 'RF_score': rf.feature_importances_})
    xgb = XGBRegressor(n_estimators=1000, random_state=7)
    xgb.fit(X, y)
    xgb_imp = pd.DataFrame({'feature': X.columns, 'XGB_score': xgb.feature_importances_})
    selector = SelectKBest(mutual_info_regression, k='all')
    selector.fit(X, y)
    skb_imp = pd.DataFrame({'feature': X.columns, 'SKB_score': selector.scores_})
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    pca = PCA(n_components=0.9999)
    pca.fit(X_scaled)
    pca_loadings = pd.DataFrame(pca.components_.T, index=X.columns).abs().sum(axis=1)
    pca_imp = pd.DataFrame({'feature': pca_loadings.index, 'PCA_score': pca_loadings.values})
    selected_fs_bic = forward_selection_with_bic(X, y)
    fs_bic_scores = {feature: 1.0 for feature in selected_fs_bic}
    fs_bic_imp = pd.DataFrame({
        'feature': X.columns,
        'FS_BIC_score': [fs_bic_scores.get(f, 0.0) for f in X.columns]  # 1.0 nếu được chọn, 0.0 nếu không
    })

    combined = rf_imp.merge(skb_imp, on='feature')\
                    .merge(pca_imp, on='feature')\
                    .merge(xgb_imp, on='feature')\
                    .merge(fs_bic_imp, on='feature')
    
    combined.fillna(1e-9, inplace=True)  # hoặc 1e-9 nếu cần

    rank_columns = []
    for method in ['RF_score', 'SKB_score', 'PCA_score', 'XGB_score', 'FS_BIC_score']:
        ascending = False
        combined[f'{method}_rank'] = combined[method].rank(ascending=ascending, method='dense')
        rank_columns.append(f'{method}_rank')
    
    combined['Weighted_Avg_Rank'] = combined[rank_columns].apply(
        lambda row: sum(row[col] * WEIGHTS.get(col, 0) for col in WEIGHTS.keys()), axis=1
    )
    with open("combined.txt", "a", encoding="utf-8") as file_out:
        file_out.write(f"\n=== combined dataframe of {target}===\n")
        file_out.write(combined.to_string())
        file_out.write(f"\n=== end of combined dataframe ===\n")

    # Chọn top features
    top_features = combined.sort_values('Weighted_Avg_Rank').head(30)['feature'].tolist()
    final_features[target] = top_features
    with open("selected_feature.txt", "a", encoding="utf-8") as f:
        f.write(f"\n{target} : {top_features}\n")

BPS_features = final_features['bps_pred']
MPS_features = final_features['mps_pred']
FPS_features = final_features['fps_pred']
HOMO_eV_features = final_features['HOMO_eV']
LUMO_eV_features = final_features['LUMO_eV']
c_v_features = final_features['c_v']

def save_model(model_name, model_type, method):
    pkl_filename = f'{model_type}_{type(model_name).__name__}_{method}.pkl'
    with open(pkl_filename, 'wb') as file:  
        pickle.dump(model_name, file)
    with open(pkl_filename, 'rb') as file:  
        saved_model = pickle.load(file)
    print(saved_model)

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

target_features2 = {
    'bps_pred': BPS_features,
    'mps_pred': MPS_features,
    'fps_pred': FPS_features
}

cv = 5

for i, target in enumerate(targets2):
    print(f"\n=== Building models for {target} ===")
    print("="*50)
    
    selected_features = target_features2[target]
    X = df2[selected_features]
    # Handle infinity values by replacing with mean
    inf_mask = X.isin([np.inf, -np.inf])
    if inf_mask.any().any():
        X = X.replace([np.inf, -np.inf], np.nan)
        X = X.fillna(X.mean())
    y = df2[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

    # 1. Pipeline và param grid cho Random Forest
    rf_pipe = Pipeline([
        ("scaler", MinMaxScaler()), 
        ('model', RandomForestRegressor(criterion='absolute_error', random_state=7))
    ])
    
    rf_param_grid = {
        "model__n_estimators": [100, 200],
        "model__max_depth": [10, 20],
    }
    
    # 2. Pipeline và param grid cho XGBoost
    xgb_pipe = Pipeline([
        ("scaler", MinMaxScaler()), 
        ('model', XGBRegressor(objective='reg:absoluteerror', random_state=7))
    ])
    
    xgb_param_grid = {
    "model__n_estimators": [5000],
    "model__max_depth": [25],
    "model__learning_rate": [0.04],
    "model__min_child_weight": [10]    # Số lượng mẫu tối thiểu ở lá
}
    
    models = [
        # ('Random Forest', rf_pipe, rf_param_grid),
        ('XGBoost', xgb_pipe, xgb_param_grid)
    ]
    
    best_models = {}
    
    for name, pipe, param_grid in models:
        print(f'\n----- Optimizing {name} for {targets2[i]} -----')
        search = GridSearchCV(pipe, param_grid, n_jobs=-1, 
                            scoring = 'neg_mean_absolute_error' )
        search.fit(X_train, y_train,   model__verbose=False       
)
        
        best_model = search.best_estimator_
        y_pred = best_model.predict(X_test)
        
        # Tính các metrics
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        
        print(f"\nBest parameters for {name}:")
        for param, value in search.best_params_.items():
            print(f"{param}: {value}")
        
        print("\nEvaluation Metrics on Test Set:")
        print(f"R2 Score: {r2:.4f}")
        print(f"RMSE: {rmse:.4f}")
        print(f"MAE: {mae:.4f}")
        print(f"MAPE: {mape:.4f}%")
        
        best_models[name] = {
            'model': best_model,
            'metrics': {
                'R2': r2,
                'RMSE': rmse,
                'MAE': mae,
                'MAPE': mape
            }
        }
    
    # So sánh kết quả tốt nhất giữa 2 mô hình
    print('\n----- Model Comparison -----')
    print(f"{'Model':<15} {'R2':>8} {'RMSE':>8} {'MAE':>8} {'MAPE':>8}")
    for name, result in best_models.items():
        metrics = result['metrics']
        print(f"{name:<15} {metrics['R2']:8.4f} {metrics['RMSE']:8.4f} " 
              f"{metrics['MAE']:8.4f} {metrics['MAPE']:8.4f}%")
    
    # Xác định best model dựa trên MAE
    best_model_name = min(best_models.items(), 
                         key=lambda x: x[1]['metrics']['MAE'])[0]
    print(f"\nBest model for {target} is {best_model_name} "
          f"(MAE = {best_models[best_model_name]['metrics']['MAE']:.4f})")
    
    # Lưu best model
    model_name = f"{targets2[i]}_{best_model_name.replace(' ', '_')}"
    save_model(best_models[best_model_name]['model'], 'regression', targets2[i])
    plt.figure(figsize=(6, 6))
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.title(f'{target} - Actual vs. Predicted')
    plt.text(
        min(y_test), 
        max(y_pred), 
        f"MAPE: {mape:.2f}%\nR²: {r2:.2f}"
    )
    plt.show()

model_paths = {
    'bps_model': 'regression_Pipeline_bps_pred.pkl',
    'mps_model': 'regression_Pipeline_mps_pred.pkl',
}
with open("selected_feature.txt", "r", encoding="utf-8") as f:
    temp_features = {}
    for line in f:
        line = line.strip()
        if not line:
            continue
        key, val = line.split(":", 1)
        temp_features[key.strip()] = ast.literal_eval(val.strip())

BPS_features = temp_features['bps_pred']
MPS_features = temp_features['mps_pred']
models = {}
for name, path in model_paths.items():
    if os.path.exists(path):
        print(f"Loading {name} from {path}")
        with open(path, 'rb') as f:
            models[name] = pickle.load(f)
    else:
        print(f"Warning: {path} does not exist")
X_bps = df1[BPS_features] 
X_mps = df1[MPS_features]  
for name, path in model_paths.items():
    if os.path.exists(path):
        print(f"Loading {name} from {path}")
        with open(path, 'rb') as f:
            models[name] = pickle.load(f)
    else:
        print(f"Warning: {path} does not exist")

# Make predictions if models were loaded successfully
if 'bps_model' in models:
    # Kiểm tra features tồn tại trong dataframe
    missing_features = [feat for feat in BPS_features if feat not in df1.columns]
    if missing_features:
        print(f"Warning: Missing features for BPS prediction: {missing_features}")
    else:
        X_bps = df1[BPS_features]
        df1['bps_predicted'] = models['bps_model'].predict(X_bps)
        print("BPS predictions added to dataframe")

if 'mps_model' in models:
    missing_features = [feat for feat in MPS_features if feat not in df1.columns]
    if missing_features:
        print(f"Warning: Missing features for MPS prediction: {missing_features}")
    else:
        X_mps = df1[MPS_features]
        df1['mps_predicted'] = models['mps_model'].predict(X_mps)
        print("MPS predictions added to dataframe")
target_features1 = {
    'HOMO_eV': HOMO_eV_features + ['bps_predicted', 'mps_predicted'],
    'LUMO_eV': LUMO_eV_features + ['bps_predicted', 'mps_predicted'],
    'c_v': c_v_features + ['bps_predicted', 'mps_predicted']
}

for i, target in enumerate(targets1):
    print(f"\n=== Building models for {target} ===")
    print("="*50)
    
    selected_features = target_features1[target]
    X = df1[selected_features]
    # Handle infinity values by replacing with mean
    inf_mask = X.isin([np.inf, -np.inf])
    if inf_mask.any().any():
        X = X.replace([np.inf, -np.inf], np.nan)
        X = X.fillna(X.mean())
    y = df1[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

    # 1. Pipeline và param grid cho Random Forest
    rf_pipe = Pipeline([
        ("scaler", MinMaxScaler()), 
        ('model', RandomForestRegressor(criterion='absolute_error', random_state=7))
    ])
    
    rf_param_grid = {
        "model__n_estimators": [100, 200],
        "model__max_depth": [10, 20],
    }
    
    # 2. Pipeline và param grid cho XGBoost
    xgb_pipe = Pipeline([
        ("scaler", MinMaxScaler()), 
        ('model', XGBRegressor(objective='reg:absoluteerror', random_state=7))
    ])
    
    xgb_param_grid = {
    "model__n_estimators": [2000],
    "model__max_depth": [20],
    "model__learning_rate": [0.01],
    "model__min_child_weight": [10]    # Số lượng mẫu tối thiểu ở lá
}
    
    models = [
        # ('Random Forest', rf_pipe, rf_param_grid),
        ('XGBoost', xgb_pipe, xgb_param_grid)
    ]
    
    best_models = {}
    
    for name, pipe, param_grid in models:
        print(f'\n----- Optimizing {name} for {targets1[i]} -----')
        search = GridSearchCV(pipe, param_grid, n_jobs=-1, 
                            scoring = 'neg_mean_absolute_error' )
        search.fit(X_train, y_train,   model__verbose=False       
)
        
        best_model = search.best_estimator_
        y_pred = best_model.predict(X_test)
        
        # Tính các metrics
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        
        print(f"\nBest parameters for {name}:")
        for param, value in search.best_params_.items():
            print(f"{param}: {value}")
        
        print("\nEvaluation Metrics on Test Set:")
        print(f"R2 Score: {r2:.4f}")
        print(f"RMSE: {rmse:.4f}")
        print(f"MAE: {mae:.4f}")
        print(f"MAPE: {mape:.4f}%")
        
        best_models[name] = {
            'model': best_model,
            'metrics': {
                'R2': r2,
                'RMSE': rmse,
                'MAE': mae,
                'MAPE': mape
            }
        }
    
    # So sánh kết quả tốt nhất giữa 2 mô hình
    print('\n----- Model Comparison -----')
    print(f"{'Model':<15} {'R2':>8} {'RMSE':>8} {'MAE':>8} {'MAPE':>8}")
    for name, result in best_models.items():
        metrics = result['metrics']
        print(f"{name:<15} {metrics['R2']:8.4f} {metrics['RMSE']:8.4f} " 
              f"{metrics['MAE']:8.4f} {metrics['MAPE']:8.4f}%")
    
    # Xác định best model dựa trên MAE
    best_model_name = min(best_models.items(), 
                         key=lambda x: x[1]['metrics']['MAE'])[0]
    print(f"\nBest model for {target} is {best_model_name} "
          f"(MAE = {best_models[best_model_name]['metrics']['MAE']:.4f})")
    
    # Lưu best model
    model_name = f"{targets1[i]}_{best_model_name.replace(' ', '_')}"
    save_model(best_models[best_model_name]['model'], 'regression', targets1[i])
    plt.figure(figsize=(6, 6))
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.title(f'{target} - Actual vs. Predicted')
    plt.text(
        min(y_test), 
        max(y_pred), 
        f"MAPE: {mape:.2f}%\nR²: {r2:.2f}"
    )
    plt.show()
    

ModuleNotFoundError: No module named 'xgboost'